In [57]:
%config Completer.use_jedi = False

In [1]:
from smtutil import hiveutil, oracle_to_hive
import pandas as pd
from pyspark.sql.functions import lag, lead, first, last, udf, lit, col, array_sort, collect_list, expr, row_number, lit,concat
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DateType, TimestampType, StructType, StructField, DoubleType, BooleanType
from pyspark.sql import functions as func
from datetime import datetime as dt
import datetime
import json
from datetime import date, timedelta, datetime
import os
import numpy as np
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)
from typing import Union

Connect to Oracle Successfully
Connect to Oracle Successfully


In [2]:
# spark.stop()
# conn.close()
# sc.stop()

In [3]:
sc, spark = hiveutil.get_sc_spark()

In [4]:
from weblog_util.common import getCleansedDomainList, getAggValue, getDomainSummary

In [5]:
conn=oracle_to_hive.connect_to_Oracle()

Connect to Oracle Successfully


In [6]:
l_formatted = getCleansedDomainList(['https://rnr.hk.chinamobile.com', 'https://rnr.hkcsl.com', 'https://rnr.smartone.com'])
print(l_formatted)

['RNR.HK.CHINAMOBILE.COM', 'RNR.HKCSL.COM', 'RNR.SMARTONE.COM']


In [8]:
sdf_tddaily_v1 = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_daily/').filter(F.col('date_id')==20230116)
sdf_tddaily_v2 = spark.read.parquet('/app/ws-ic/hive/weblog_v2.db/t_domain_daily/').filter(F.col('date_id')==20230116)

sdf_raw_ssl = spark.read.csv('/app/sna_bdmrdev03/ws-simon/data/weblog_testing_20230115/', sep='|')
sdf_ssl = spark.read.parquet('/app/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/').filter(F.col('date_id')==20230116)

sdf_tddict_v1 = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_dictionary/')
sdf_tddict_v2 = spark.read.parquet('/app/ws-ic/hive/weblog_v2.db/t_domain_dictionary/')

sdf_tdi_v1 = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_index/')
sdf_tdi_v2 = spark.read.parquet('/app/ws-ic/hive/weblog_v2.db/t_domain_index/')

In [11]:
# stage 1
sdf_raw_ssl.limit(2).toPandas()

_c0            _c1             _c2             _c3   _c4   _c5  \
0  85251113001  10.177.84.137  20230115170741   18.166.42.142  5229  3292   
1  85251113001  10.177.84.137  20230115170742  101.78.207.157  2382  1514   

                              _c6             _c7  
0  report.hkjc.glassboxdigital.io  20230115170748  
1                 common.hkjc.com  20230115170749

In [12]:
# stage 2
sdf_ssl.limit(2).toPandas()

subr_num        ssl_cert_domain  byte_count  hour_binary  total_hit  \
0  19100026043       ADS.PUBMATIC.COM       44884           96          3   
1  19100026043  APNX-MATCH.DOTOMI.COM       18306           64          2   

    date_id  
0  20230116  
1  20230116

In [26]:
# stage 3
# t_doamin_index
print(sdf_tdi_v1.count(), sdf_tdi_v2.count(), sdf_tdi_v1.select(F.col('domain_index')).distinct().count(), sdf_tdi_v2.select(F.col('domain_index')).distinct().count())
sdf_tdi_v1.limit(2).toPandas()

147086 3874099 147086 3874099


domain  o_domain_index  domain_index
0  200VOD-ADAPTIVE.AKAMAIZED.NET               4             4
1  AMP-RFI-FR.CDN.AMPPROJECT.ORG              19            19

In [27]:
# t_doamin_dictonary
print(sdf_tddict_v1.count(), sdf_tddict_v2.count(), sdf_tddict_v1.select(F.col('agg_value')).distinct().count(), sdf_tddict_v2.select(F.col('agg_value')).distinct().count())
sdf_tddict_v1.limit(2).toPandas()

967868 3874797 967170 3874099


tag_gp             tag               domain agg_level  agg_value
0  <Unclassified>  <Unclassified>       WWW.GOOGLE.COM    domain          1
1  <Unclassified>  <Unclassified>  PLAY.GOOGLEAPIS.COM    domain          2

In [33]:
# join dict index
sdf_tddict_v1_tdi_v1 = sdf_tddict_v1.join(sdf_tdi_v1, 'domain', 'inner')
sdf_tddict_v2_tdi_v2 = sdf_tddict_v2.join(sdf_tdi_v2, 'domain', 'inner')

In [34]:
sdf_tddict_v1_tdi_v1 = sdf_tddict_v1_tdi_v1.withColumn('diff', F.when(F.col('agg_value')!=F.col('domain_index'),1).otherwise(0))
sdf_tddict_v2_tdi_v2 = sdf_tddict_v2_tdi_v2.withColumn('diff', F.when(F.col('agg_value')!=F.col('domain_index'),1).otherwise(0))

In [46]:
sdf_tddict_v1_tdi_v1.filter((F.col('diff')== 1)).limit(5).toPandas()

domain          tag_gp             tag agg_level  \
0               WWW.GOOGLE.COM  <Unclassified>  <Unclassified>    domain   
1          PLAY.GOOGLEAPIS.COM  <Unclassified>  <Unclassified>    domain   
2  GOOGLEADS.G.DOUBLECLICK.NET  <Unclassified>  <Unclassified>    domain   
3           WWW.GOOGLEAPIS.COM  <Unclassified>  <Unclassified>    domain   
4          APP-MEASUREMENT.COM  <Unclassified>  <Unclassified>    domain   

   agg_value  o_domain_index  domain_index  diff  
0          1           36833         36833     1  
1          2           21213         21213     1  
2          3           23609         23609     1  
3          4           25539         25539     1  
4          6           10998         10998     1

In [43]:
sdf_tddict_v1_tdi_v1.filter((F.col('diff')== 0)).limit(5).toPandas()

domain        tag_gp                tag agg_level  agg_value  \
0  B-API.FACEBOOK.COM  Social Media  Social Networking    domain         54   

   o_domain_index  domain_index  diff  
0              54            54     0

In [44]:
sdf_tddict_v2_tdi_v2.filter((F.col('diff')== 0)).limit(5).toPandas()

domain          tag_gp  \
0                            B-API.FACEBOOK.COM    Social Media   
1   W3EE98D871-GI3DGNJZG4ZTOMB.SNR.WHATSAPP.NET  <Unclassified>   
2   W445C44301-GE3TKNBQHA2DMMA.SNR.WHATSAPP.NET  <Unclassified>   
3  W46C03B7D1-GM3TCOBZGIZTOMJV.SNR.WHATSAPP.NET  <Unclassified>   
4  W4B2A92D61-GM4DKNRQGA2TGMBW.SNR.WHATSAPP.NET  <Unclassified>   

                 tag agg_level  agg_value  o_domain_index  domain_index  diff  
0  Social Networking    domain         54            54.0            54     0  
1     <Unclassified>    domain     967171             NaN        967171     0  
2     <Unclassified>    domain     967172             NaN        967172     0  
3     <Unclassified>    domain     967173             NaN        967173     0  
4     <Unclassified>    domain     967174             NaN        967174     0

In [49]:
# stage 4
# t doamin daily
print(sdf_tddaily_v1.count(), sdf_tddaily_v2.count())
sdf_tddaily_v1.limit(5).toPandas()

244587059 249951956


subr_num  agg_value agg_level  in_byte  out_byte  byte_count  total_hit  \
0  19100026043      53716    domain        0         0       16068          2   
1     94000202      49931    domain        0         0        7931          1   
2  19100026043      56758    domain        0         0       85499         10   
3     94000202      37009    domain        0         0       65044          5   
4  19100026043      77804    domain        0         0       20033          2   

   duration   date_id  
0         0  20230116  
1         0  20230116  
2         0  20230116  
3         0  20230116  
4         0  20230116

In [52]:
sdf_tddaily_v1.describe().toPandas()

summary              subr_num           agg_value  agg_level    in_byte  \
0   count             244587059           244587059  244587059  244587059   
1    mean   4.191580546823043E8  43991.911797336754       None        0.0   
2  stddev  1.5081756041370733E9  26520.261110498126       None        0.0   
3     min              19590051                   1     domain          0   
4     max          450000145912              146767     domain          0   

    out_byte           byte_count           total_hit   duration      date_id  
0  244587059            244587059           244587059  244587059    244587059  
1        0.0    1379259.399242153   8.180515519424926        0.0  2.0230116E7  
2        0.0  5.882379319892411E7  195.54210262736296        0.0          0.0  
3          0                  374                   1          0     20230116  
4          0         527111807460              663103          0     20230116

In [53]:
sdf_mth = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_monthly/month_id=202212/')

In [54]:
sdf_mth.limit(5).toPandas()

subr_num  agg_value agg_level  in_byte  out_byte  byte_count  total_hit  \
0  19590481      67442    domain        0         0      177069         10   
1  97259757      69824    domain        0         0      987380        101   
2  19590597      24118    domain        0         0       18568          1   
3  97259757      71620    domain        0         0      687726         65   
4  19590597      31959    domain        0         0     4448291        230   

   duration  days  
0         0     9  
1         0    19  
2         0     1  
3         0    26  
4         0    18

In [31]:
# sdf_tdi_v1.select(F.col('domain_index')).describe().show()

In [29]:
# sdf_v1.groupby(F.col('agg_value')).agg(F.countDistinct('agg_value').alias('cnt_d')).count()
# sdf_v2.groupby(F.col('agg_value')).agg(F.countDistinct('agg_value').alias('cnt_d')).count()

In [83]:
sdf_dict_join_index = sdf_agg_value_table_v1.join(sdf_domain, 'domain', 'inner')

In [69]:
sdf_agg_value_table_v1.count(), sdf_domain.count()

(967868, 146767)

In [84]:
sdf_dict_join_index2 = sdf_dict_join_index.withColumn('diff', F.when(F.col('agg_value')!=F.col('domain_index'),1).otherwise(0))

In [89]:
sdf_agg_value_table_v1.count(), sdf_domain.count(), sdf_dict_join_index2.count()

(967868, 146767, 118896)

In [91]:
sdf_dict_join_index2.select(F.sum('diff')).show()

+---------+
|sum(diff)|
+---------+
|   118895|
+---------+



In [93]:
sdf_dict_join_index2.filter((F.col('diff')== 1)).limit(20).toPandas()

domain          tag_gp             tag agg_level  \
0                   WWW.GOOGLE.COM  <Unclassified>  <Unclassified>    domain   
1              PLAY.GOOGLEAPIS.COM  <Unclassified>  <Unclassified>    domain   
2      GOOGLEADS.G.DOUBLECLICK.NET  <Unclassified>  <Unclassified>    domain   
3               WWW.GOOGLEAPIS.COM  <Unclassified>  <Unclassified>    domain   
4              APP-MEASUREMENT.COM  <Unclassified>  <Unclassified>    domain   
5                  WWW.GSTATIC.COM  <Unclassified>  <Unclassified>    domain   
6    PAGEAD2.GOOGLESYNDICATION.COM  <Unclassified>  <Unclassified>    domain   
7         WWW.GOOGLEADSERVICES.COM  <Unclassified>  <Unclassified>    domain   
8              STATIC.WHATSAPP.NET  <Unclassified>  <Unclassified>    domain   
9        TPC.GOOGLESYNDICATION.COM  <Unclassified>  <Unclassified>    domain   
10                MTALK.GOOGLE.COM  <Unclassified>  <Unclassified>    domain   
11       LH3.GOOGLEUSERCONTENT.COM  <Unclassified>  <Unclassified>    domain   
12            FONTS.GOOGLEAPIS.COM  <Unclassified>  <Unclassified>    domain   
13               WWW.GOOGLE.COM.HK  <Unclassified>  <Unclassified>    domain   
14            ADSERVICE.GOOGLE.COM  <Unclassified>  <Unclassified>    domain   
15               FONTS.GSTATIC.COM  <Unclassified>  <Unclassified>    domain   
16  SECUREPUBADS.G.DOUBLECLICK.NET  <Unclassified>  <Unclassified>    domain   
17                 WWW.YOUTUBE.COM  <Unclassified>  <Unclassified>    domain   
18         ADSERVICE.GOOGLE.COM.HK  <Unclassified>  <Unclassified>    domain   
19                MMG.WHATSAPP.NET  <Unclassified>  <Unclassified>    domain   

    agg_value  o_domain_index  domain_index  diff  
0           1           36833         36833     1  
1           2           21213         21213     1  
2           3           23609         23609     1  
3           4           25539         25539     1  
4           6           10998         10998     1  
5           7           20104         20104     1  
6           8           36688         36688     1  
7          11            8880          8880     1  
8          14            2508          2508     1  
9          15            2963          2963     1  
10         17           74163         74163     1  
11         18            5748          5748     1  
12         19           24884         24884     1  
13         20           57144         57144     1  
14         22           53736         53736     1  
15         23           65313         65313     1  
16         24           67166         67166     1  
17         26           43570         43570     1  
18         27           32693         32693     1  
19         28           52187         52187     1

In [94]:
sdf_dict_join_index2.groupby(F.col('domain')).agg(F.countDistinct('domain_index').alias('cnt_d')).filter(F.col('cnt_d') > 1).count()

0

In [49]:
sdf_agg_value_table = sdf_agg_value_table_v1.filter(F.col('domain').isin(l_formatted))
sdf_agg_value_table.toPandas()

domain  o_domain_index  domain_index
0           RNR.HKCSL.COM           50985         50985
1  RNR.HK.CHINAMOBILE.COM            1602          1602
2        RNR.SMARTONE.COM           30934         30934

In [53]:
sdf_v1_joined = sdf_v1.join(sdf_agg_value_table, sdf_agg_value_table.domain_index==sdf_v1.agg_value ,'inner')
sdf_v2_joined = sdf_v2.join(sdf_agg_value_table, sdf_agg_value_table.domain_index==sdf_v2.agg_value ,'inner')

In [54]:
sdf_v1_joined.count(), sdf_v2_joined.count()

(1057, 1057)

In [14]:
sdf_v1_joined.toPandas()

Empty DataFrame
Columns: [agg_value, subr_num, agg_level, in_byte, out_byte, byte_count, total_hit, duration, date_id, tag_gp, tag, domain, agg_level]
Index: []

In [13]:
sdf_v2_joined.toPandas()

agg_value    subr_num agg_level  in_byte  out_byte  byte_count  total_hit  \
0     903320    98496427    domain        0         0       14630          1   
1     880645    61810402    domain        0         0       13556          1   
2     903116    92277972    domain        0         0      185978          1   
3     903116    67003000    domain        0         0      543349          7   
4     903116  1994320186    domain        0         0       26212          1   

   duration   date_id          tag_gp             tag                  domain  \
0         0  20230116  <Unclassified>  <Unclassified>           RNR.HKCSL.COM   
1         0  20230116  <Unclassified>  <Unclassified>  RNR.HK.CHINAMOBILE.COM   
2         0  20230116  <Unclassified>  <Unclassified>        RNR.SMARTONE.COM   
3         0  20230116  <Unclassified>  <Unclassified>        RNR.SMARTONE.COM   
4         0  20230116  <Unclassified>  <Unclassified>        RNR.SMARTONE.COM   

  agg_level  
0    domain  
1    domain  
2    domain  
3    domain  
4    domain

In [29]:
sdf_raw.limit(5).toPandas()

subr_num                                    ssl_cert_domain  byte_count  \
0  19100026043                                   ADS.PUBMATIC.COM       44884   
1  19100026043                              APNX-MATCH.DOTOMI.COM       18306   
2  19100026043                             EUS.RUBICONPROJECT.COM       21798   
3  19100026043  MESSAGING.PROD.AP-EAST-1.VAM-HK.CLOUD1.VV1865.COM        9268   
4  19100036073                 MICROSOFTEDGEWELCOME.MICROSOFT.COM       50858   

   hour_binary  total_hit   date_id  
0           96          3  20230116  
1           64          2  20230116  
2           32          1  20230116  
3           32          1  20230116  
4       524288          4  20230116

In [16]:
sdf_raw_joined = sdf_raw.filter(F.col('ssl_cert_domain').isin(l_formatted))

In [17]:
sdf_raw_joined.count()

1057

In [18]:
sdf_raw_joined.limit(5).toPandas()

subr_num   ssl_cert_domain  byte_count  hour_binary  total_hit   date_id
0  68590444     RNR.HKCSL.COM     1108161          256          6  20230116
1  94464025  RNR.SMARTONE.COM      854464        32768          5  20230116
2  94546553  RNR.SMARTONE.COM     4379168            1          8  20230116
3  97231655  RNR.SMARTONE.COM       42310           64          2  20230116
4  98494956  RNR.SMARTONE.COM      135023           64          5  20230116

In [21]:
sdf_temp = sdf_raw_joined.filter(F.col('subr_num').isin(list(sdf_v2_joined.toPandas()['subr_num'])))

In [28]:
sdf_raw_joined.filter(F.col('subr_num')=='98496427').toPandas()

Empty DataFrame
Columns: [subr_num, ssl_cert_domain, byte_count, hour_binary, total_hit, date_id]
Index: []

In [34]:
date_id = 20230116
sdf_raw2_joined = spark.sql(f"""
    select
        subr_num,
        domain_index as agg_value,
        'domain' as agg_level,
        0 as in_byte,
        0 as out_byte,
        byte_count,
        total_hit,
        0 as duration,
        date_id
    from prd_biz_summ_vw.vw_idp_ssl_sed_call_summ
    join weblog_v2.t_domain_index
        on vw_idp_ssl_sed_call_summ.ssl_cert_domain = t_domain_index.domain
    where date_id = {date_id}
    """.format(
        date_id=date_id
    ))

In [39]:
sdf_domain = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_index/')
sdf_domain2 = spark.read.parquet('/app/ws-ic/hive/weblog_v2.db/t_domain_index/')

In [40]:
sdf_domain.count(), sdf_domain2.count()

(146767, 3874099)

In [43]:
sdf_agg_value_table_v1.count(), sdf_agg_value_table_v2.count()

(967868, 3874797)

In [46]:
sdf_domain2.filter(F.col('domain').isin(l_formatted)).toPandas()

domain  o_domain_index  domain_index
0  RNR.HK.CHINAMOBILE.COM            1602          1602
1           RNR.HKCSL.COM           50985         50985
2        RNR.SMARTONE.COM           30934         30934

In [47]:
sdf_agg_value_table_v2.filter(F.col('domain').isin(l_formatted)).toPandas()

tag_gp             tag                  domain agg_level  agg_value
0  <Unclassified>  <Unclassified>           RNR.HKCSL.COM    domain     903320
1  <Unclassified>  <Unclassified>        RNR.SMARTONE.COM    domain     903116
2  <Unclassified>  <Unclassified>  RNR.HK.CHINAMOBILE.COM    domain     880645

In [42]:
sdf_domain2.limit(5).toPandas()

domain  o_domain_index  domain_index
0                    BATS.HDIKE.COM              57            57
1                     BRYNH-LEI.COM              63            63
2                  CDM.CONNATIX.COM              67            67
3                     IMG.999992.CO             170           170
4  R1---SN-QXO7RN7K.GOOGLEVIDEO.COM             274           274

In [38]:
# sdf_raw2_joined_filter = sdf_raw2_joined.join(sdf_agg_value_table, 'agg_value', 'inner')

In [14]:
sdf_agg_value_table_v1.filter(F.col('domain').isin(l_formatted)).toPandas()

tag_gp             tag                  domain agg_level  agg_value
0  <Unclassified>  <Unclassified>           RNR.HKCSL.COM    domain     903320
1  <Unclassified>  <Unclassified>        RNR.SMARTONE.COM    domain     903116
2  <Unclassified>  <Unclassified>  RNR.HK.CHINAMOBILE.COM    domain     880645

In [15]:
sdf_agg_value_table_v2.filter(F.col('domain').isin(l_formatted)).toPandas()

tag_gp             tag                  domain agg_level  agg_value
0  <Unclassified>  <Unclassified>           RNR.HKCSL.COM    domain     903320
1  <Unclassified>  <Unclassified>        RNR.SMARTONE.COM    domain     903116
2  <Unclassified>  <Unclassified>  RNR.HK.CHINAMOBILE.COM    domain     880645

In [12]:
sdf_v1.join(sdf_agg_value_table_v1, 'agg_value', 'inner')

DataFrame[agg_value: int, subr_num: bigint, agg_level: string, in_byte: int, out_byte: int, byte_count: bigint, total_hit: bigint, duration: int, date_id: int, tag_gp: string, tag: string, domain: string, agg_level: string]

In [8]:
sdf_v1.limit(1).toPandas()

subr_num  agg_value agg_level  in_byte  out_byte  byte_count  total_hit  \
0  19100026043      53716    domain        0         0       16068          2   

   duration   date_id  
0         0  20230116

## Get filter table

In [ ]:
# # source_table = pd.read_csv('./game_and_music_and_football_lover_with_membership_gender.csv')
# source_table = pd.read_csv('./game_and_music_and_football_lover_with_membership_gender.csv', converters = {'SUBR_NUM': str})

# # PREPROCESSING
# source_table = source_table[source_table['SEGMENT_TYPE']=='Narrowband']
# source_table['CUST_NUM'] = source_table['CUST_NUM'].apply(lambda x: '0' * (8 - len(str(x))) + str(x) if (8 - len(str(x))) > 0 else str(x))

In [ ]:
# filter_table = spark.createDataFrame(source_table[['CUST_NUM', 'SUBR_NUM']])

In [113]:
df_rnr = pd.read_sql_query("""
select * from PRD_BIZ_SUMM_VW.VW_RNR_CONTACT""",con=conn)

In [114]:
df_rnr_raw = df_rnr.copy()

In [12]:
df_mapped = pd.read_csv("./rnr_websites_mapped2.csv", header=0)

In [15]:
df_rnr.head()

RNR_CONTACT_ID                 ICCID  CUST_NUM    MSISDN SIM_LICENSEE  \
0        A-8RKF7  89852060607142047528  41269934  91374003          SMC   
1        A-38C34  89852061208236065085  41643084  60105242          SMC   
2        A-3AE71  89852061208240275993  07295672  92267719          SMC   
3        A-23IPZ  89852061209230402381  12690154  98689890          SMC   
4        A-ADV55  89852061209231787046  07120762  67701883          SMC   

        SIM_TYPE         IDBR ID_TYPE LAST_NAME_CHI FIRST_NAME_CHI LAST_NAME  \
0  Local Prepaid   E248316(9)       I                          ???             
1  Local Prepaid  WX030832(1)       I                                          
2       Postpaid   Z652486(A)       I             ?             ??        Li   
3       Postpaid   K877144(8)       I             ?             ??        Li   
4       Postpaid   Z403198(A)       I             ?             ??    Cheung   

      FIRST_NAME                BIRTH TITLE      STATUS PENDING  \
0    YIM SAU KAM  1948-11-03 00:00:00   Mrs  registered       N   
1       APRIATIN  1984-04-02 00:00:00    Ms  registered       N   
2      Siu Chung  1985-06-22 00:00:00    Mr  registered       N   
3  Lok Man Ricky  1978-10-03 00:00:00    Mr  registered       N   
4        Man Kin  1980-04-24 00:00:00    Mr  registered       N   

  VERIFICATION_METHOD    VERIFICATION_DATE    VERIFIED_BY REGISTRATION_TYPE  \
0                 OCR  2022-08-27 00:00:00  RNRportal-001             adult   
1                 OCR  2022-04-17 00:00:00  RNRportal-001             adult   
2                 FTF  2022-04-18 00:00:00          NTS07             adult   
3                 FTF  2022-03-14 00:00:00       MonaChoi             adult   
4                 FTF  2022-10-18 00:00:00          TKP11             adult   

     REGISTRATION_DATE  DEREGISTRATION_DATE DEREGISTERED_BY REJECT_REASON  \
0  2022-08-27 00:00:00  2999-12-31 00:00:00                                 
1  2022-04-17 00:00:00  2999-12-31 00:00:00                                 
2  2022-04-18 00:00:00  2999-12-31 00:00:00                                 
3  2022-03-14 00:00:00  2999-12-31 00:00:00                                 
4  2022-10-18 00:00:00  2999-12-31 00:00:00                                 

  GUARANTOR_ID_NUMBER GUARANTOR_ID_TYPE GUARANTOR_LAST_NAME_CHI  \
0                                                                 
1                                                                 
2                                                                 
3                                                                 
4                                                                 

  GUARANTOR_FIRST_NAME_CHI GUARANTOR_LAST_NAME GUARANTOR_FIRST_NAME  \
0                                                                     
1                                                                     
2                                                                     
3                                                                     
4                                                                     

       GUARANTOR_BIRTH GUARANTOR_TITLE CREATED_DATE LAST_UPDATED_DATE  \
0  1900-01-01 00:00:00                   2022-08-27        2022-08-27   
1  1900-01-01 00:00:00                   2022-04-17        2022-04-17   
2  1900-01-01 00:00:00                   2022-04-18        2022-04-18   
3  1900-01-01 00:00:00                   2022-03-14        2022-03-14   
4  1900-01-01 00:00:00                   2022-10-18        2022-10-18   

      CREATED_BY LAST_UPDATED_BY             CONTACT_EMAIL CONTACT_NUMBER  \
0  RNRportal-001   RNRportal-001                                            
1  RNRportal-001   RNRportal-001    deryazeputry@gmail.com       60105242   
2          NTS07           NTS07  kittyeepiyo@yahoo.com.hk       92267719   
3       MonaChoi        MonaChoi       ricky8989@gmail.com       98689890   
4          TKP11           TKP11      mosungsung@gmail.com       94483400  

In [218]:
print(df_rnr.shape)
df_rnr = df_rnr[(df_rnr['STATUS']=='registered') & (df_rnr['EXISTING']=='Y')]
print(df_rnr.shape)

(444987, 63)
(102684, 63)


In [219]:
# df_rnr[['EXISTING', 'STATUS']].count_value()
print(df_rnr['EXISTING'].value_counts())
print(df_rnr['STATUS'].value_counts())

Y    102684
Name: EXISTING, dtype: int64
registered    102684
Name: STATUS, dtype: int64


In [220]:
df_cust_to_HKID = df_rnr[['CUST_NUM', 'MSISDN', 'IDBR']]
df_cust_to_HKID.rename(columns={'IDBR':'D_HKID_N', 'MSISDN':'SUBR_NUM'}, inplace=True)
# df_cust_to_HKID.set_index('CUST_NUM', inplace=True)

In [221]:
df_cust_to_HKID.head()

CUST_NUM  SUBR_NUM     D_HKID_N
0  41269934  91374003   E248316(9)
1  41643084  60105242  WX030832(1)
7  41674402  96874935  WX007968(3)
8  41762576  64653070  WX208638(5)
9  41396267  96554061   K489711(0)

In [222]:
df_cust_to_HKID.shape

(102684, 3)

## check duplicates

In [223]:
df_temp = df_rnr.groupby(['CUST_NUM', 'MSISDN'])['IDBR'].agg(['count', 'nunique'])

In [224]:
df_temp = df_rnr.groupby(['MSISDN'])['IDBR'].agg(['count', 'nunique'])

In [229]:
df_ctid = df_temp[df_temp['nunique']>1]['nunique'].reset_index()
df_ctid.shape

(0, 2)

In [231]:
l_multi_hkid = df_ctid['MSISDN']

In [230]:
df_rnr[df_rnr['MSISDN']=='95118258']

Empty DataFrame
Columns: [RNR_CONTACT_ID, ICCID, CUST_NUM, MSISDN, SIM_LICENSEE, SIM_TYPE, IDBR, ID_TYPE, LAST_NAME_CHI, FIRST_NAME_CHI, LAST_NAME, FIRST_NAME, BIRTH, TITLE, STATUS, PENDING, VERIFICATION_METHOD, VERIFICATION_DATE, VERIFIED_BY, REGISTRATION_TYPE, REGISTRATION_DATE, DEREGISTRATION_DATE, DEREGISTERED_BY, REJECT_REASON, GUARANTOR_ID_NUMBER, GUARANTOR_ID_TYPE, GUARANTOR_LAST_NAME_CHI, GUARANTOR_FIRST_NAME_CHI, GUARANTOR_LAST_NAME, GUARANTOR_FIRST_NAME, GUARANTOR_BIRTH, GUARANTOR_TITLE, CREATED_DATE, LAST_UPDATED_DATE, CREATED_BY, LAST_UPDATED_BY, CONTACT_EMAIL, CONTACT_NUMBER, BRANCH_NAME, BRANCH_NAME_CHI, BIRTH_YEAR_ONLY, GUARANTOR_BIRTH_YEAR_ONLY, NOMINATION, INITIAL_INFO, RELATIONSHIP, DUCKIE, LANGUAGE, HELD, HELD_BY, HELD_DATE, EXISTING, NO_HKID, NO_OTHER_ID_DOC, SENT_EMAIL, SENT_SMS, CREATE_TS, REFRESH_TS, SOURCE_INDICATOR, PROMOTOR_CODE, ID_COPY_FLG, GUARANTOR_ID_COPY_FLG, ID_COPY_FLG_IMAGE, GUARANTOR_ID_COPY_FLG_IMAGE]
Index: []

In [47]:
rnr_cust_list = list(df_mapped[df_mapped['CUST_NUM']!]['CUST_NUM'])

## PostPaid Master

In [88]:
df_postpaid_mas_n1 = pd.read_sql_query("""
select
                VW_PREPD_POSTPAID_SUBR_N1.CUST_NUM,
                VW_PREPD_POSTPAID_SUBR_N1.SUBR_NUM,
                VW_PREPD_POSTPAID_SUBR_N1.D_HKID_N

from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1
left join PRD_BIZ_SUMM_VW.VW_RATE_PLAN_REF
on VW_PREPD_POSTPAID_SUBR_N1.Rate_Plan_Cd = VW_RATE_PLAN_REF.Rate_Plan_Cd

left join PRD_BIZ_SUMM_VW.VW_PREPD_DEMO_CARD
on VW_PREPD_POSTPAID_SUBR_N1.Subr_Num = VW_PREPD_DEMO_CARD.Msisdn

left join PRD_BIZ_SUMM_VW.VW_SUBR_NOMINATION_HIST 
on VW_PREPD_POSTPAID_SUBR_N1.Cust_Num = VW_SUBR_NOMINATION_HIST.CUST_NUM
and VW_PREPD_POSTPAID_SUBR_N1.subr_num = VW_SUBR_NOMINATION_HIST.subr_num

left join PRD_BIZ_SUMM_VW.VW_SUBR_DATA_ENTITLE
on VW_PREPD_POSTPAID_SUBR_N1.Cust_Num = VW_SUBR_DATA_ENTITLE.CUST_NUM
and  VW_PREPD_POSTPAID_SUBR_N1.subr_num = VW_SUBR_DATA_ENTITLE.subr_num

left join PRD_BIZ_SUMM_VW.VW_SHK_RATE_PLAN_GRP_REF
on VW_SUBR_DATA_ENTITLE.FREE_DATA_ENTITLE = VW_SHK_RATE_PLAN_GRP_REF.FREE_DATA_ENTITLE

WHERE VW_PREPD_POSTPAID_SUBR_N1.D_Active_Subr_Flg = 'Y'
                AND VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type = 'PO'
                AND CASE 
                                WHEN (
                                                                VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type <> 'PO'
                                                                AND VW_PREPD_DEMO_CARD.Msisdn <> ' '
                                                                )
                                                THEN 'N'
                                WHEN (
                                                                (coalesce(VW_RATE_PLAN_REF.Rate_Plan_Grp, ' ')) IN (
                                                                                'Add-on Numbers'
                                                                                ,'Non Revenue Mobile Plan'
                                                                                ,'Non Revenue BB Line'
                                                                                ,'DHL Special Project'
                                                                                )
                                                                )
                                                THEN 'N'
                                WHEN (
                                                                (coalesce(VW_PREPD_POSTPAID_SUBR_N1.Cust_Type_Cd, ' ')) IN (
                                                                                'SMC'
                                                                                ,'LPUC1'
                                                                                ,'HKBU1'
                                                                                ,'RWSU1'
                                                                                ,'HKBU2'
                                                                                ,'GCU1'
                                                                                )
                                                                )
                                                THEN 'N'
                                ELSE 'Y'
                                END = 'Y'
                AND VW_PREPD_POSTPAID_SUBR_N1.NC_FLG = 'N'
                AND VW_SHK_RATE_PLAN_GRP_REF.SHK_PLAN_SUBGRP NOT IN (
                                'Excluded'
                                ,'MVNOs (Local)'
                                ,'MVNOs (Roaming)'
                                )
                AND VW_SUBR_NOMINATION_HIST.NOMINATION_FLG = 'N'
""",con=conn)

In [89]:
df_postpaid_mas_n1.head(5)

CUST_NUM  SUBR_NUM    D_HKID_N
0  06513395  91332807  Z013769(4)
1  11857893  96858977  R292387(3)
2  03691856  91091020  K259865(5)
3  11778283  96992982  Z960306(A)
4  09813707  63910870  Z944401(8)

In [134]:
df_postpaid_mas_n1_all = pd.read_sql_query("""
select
                *

from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1

WHERE VW_PREPD_POSTPAID_SUBR_N1.D_Active_Subr_Flg = 'Y'
    AND VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type = 'PO'
""",con=conn)
# df_postpaid_mas_n1_all = pd.read_sql_query("""
# select
#                 VW_PREPD_POSTPAID_SUBR_N1.CUST_NUM,
#                 VW_PREPD_POSTPAID_SUBR_N1.SUBR_NUM,
#                 VW_PREPD_POSTPAID_SUBR_N1.D_HKID_N

# from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1

# WHERE VW_PREPD_POSTPAID_SUBR_N1.D_Active_Subr_Flg = 'Y'
#     AND VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type = 'PO'
# """,con=conn)

In [123]:
df_rnr_ver3 = pd.read_csv('/app/ws-simon/projects/pre_post_competitor_20221212/rnr_websites_mapped3.csv')
df_rnr_ver3.head(1)

SUBR_NUM       MSISDN ACCESS_DATE         SSL_CERT_DOMAIN  BYTE_COUNT  \
0  52862396  85252862396  2022-09-11  RNR.HK.CHINAMOBILE.COM   1431997.0   

     KB_COUNT               HOUR_BINARY  TOTAL_HIT  BYTE_COUNT_UP  \
0  1398.43457  000000000000000001000000         13       844079.0   

   KB_COUNT_UP  BYTE_COUNT_DOWN  KB_COUNT_DOWN  SECONDS   MINUTES  \
0   824.295898         587918.0     574.138672      103  1.716667   

  CUST_NUM_rnr_oracle D_HKID_N_rnr_oracle  CUST_NUM_n1_prepost  \
0                 NaN                 NaN            8620092.0   

  D_HKID_N_n1_prepost PREPOST_TYPE  
0          Z258049(8)           PO

In [124]:
df_rnr_nohkid = df_rnr_ver3[df_rnr_ver3['D_HKID_N_rnr_oracle'].isna()&df_rnr_ver3['D_HKID_N_n1_prepost'].isna()]
df_rnr_nohkid.head(1)

SUBR_NUM       MSISDN ACCESS_DATE         SSL_CERT_DOMAIN  BYTE_COUNT  \
1  64466617  85264466617  2022-09-11  RNR.HK.CHINAMOBILE.COM      6057.0   

   KB_COUNT               HOUR_BINARY  TOTAL_HIT  BYTE_COUNT_UP  KB_COUNT_UP  \
1  5.915039  000000000010000000000000          1         2076.0     2.027344   

   BYTE_COUNT_DOWN  KB_COUNT_DOWN  SECONDS   MINUTES CUST_NUM_rnr_oracle  \
1           3981.0       3.887695      305  5.083333                 NaN   

  D_HKID_N_rnr_oracle  CUST_NUM_n1_prepost D_HKID_N_n1_prepost PREPOST_TYPE  
1                 NaN                  NaN                 NaN          NaN

In [130]:
df_check = df_postpaid_mas_n1_all[df_postpaid_mas_n1_all['SUBR_NUM'].isin(df_rnr_nohkid['SUBR_NUM'].astype(str).tolist())]

In [133]:
df_check.head()

CUST_NUM    SUBR_NUM    D_HKID_N
1258  08660003    90124490    07290317
1265  12695741    52362380    38770895
1822  12646152  1994394577  D588752(7)
2081  12806693  1994405629  K647906(5)
2087  12642066  1994395540  P716508(4)

In [108]:
df_prepaid_mas_n1[df_prepaid_mas_n1['SUBR_NUM'].isin(['64466617', '98469591', '55060328'])]

Empty DataFrame
Columns: [CUST_NUM, SUBR_NUM, D_HKID_N]
Index: []

In [97]:
df_postpaid_mas_n1[df_postpaid_mas_n1['SUBR_NUM']=='64466617']

Empty DataFrame
Columns: [CUST_NUM, SUBR_NUM, D_HKID_N]
Index: []

In [96]:
df_prepaid_mas_n1[df_prepaid_mas_n1['SUBR_NUM']=='64466617']

Empty DataFrame
Columns: [CUST_NUM, SUBR_NUM, D_HKID_N]
Index: []

In [57]:
print(df_postpaid_mas_n1.shape)
len(df_postpaid_mas_n1['SUBR_NUM'].unique())

(1036750, 3)


1036718

In [58]:
df_postpaid_mas_n1.drop_duplicates(inplace=True)

In [59]:
print(df_postpaid_mas_n1.shape)
len(df_postpaid_mas_n1['SUBR_NUM'].unique())

(1036718, 3)


1036718

## Prepaid Master

In [90]:
df_prepaid_mas_n1 = pd.read_sql_query("""
select
                VW_PREPD_POSTPAID_SUBR_N1.CUST_NUM,
                VW_PREPD_POSTPAID_SUBR_N1.SUBR_NUM,
                VW_PREPD_POSTPAID_SUBR_N1.D_HKID_N

from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1

where 
    VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type = 'PR'
    AND VW_PREPD_POSTPAID_SUBR_N1.SUBR_STAT_CD = 'A1'
""",con=conn)

# df_prepaid_mas_n1 = pd.read_sql_query("""
# select
#                 VW_PREPD_POSTPAID_SUBR_N1.CUST_NUM,
#                 VW_PREPD_POSTPAID_SUBR_N1.SUBR_NUM,
#                 VW_PREPD_POSTPAID_SUBR_N1.D_HKID_N

# from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1
                
# left join PRD_BIZ_SUMM_VW.VW_RATE_PLAN_REF
# on VW_PREPD_POSTPAID_SUBR_N1.Rate_Plan_Cd = VW_RATE_PLAN_REF.Rate_Plan_Cd

# left join PRD_BIZ_SUMM_VW.VW_PREPD_DEMO_CARD
# on VW_PREPD_POSTPAID_SUBR_N1.Subr_Num = VW_PREPD_DEMO_CARD.Msisdn

# """,con=conn)

In [91]:
df_prepaid_mas_n1.head()

CUST_NUM   SUBR_NUM  D_HKID_N
0  50193721  196851676  00000000
1  50269111  199306890  00000000
2  50286638  199324053  00000000
3  50323712  199357971  00000000
4  50326118  199360362  00000000

In [62]:
print(df_prepaid_mas_n1.shape)
len(df_prepaid_mas_n1['SUBR_NUM'].unique())

(1757527, 3)


1757527

In [63]:
df_prepaid_mas_n1.drop_duplicates(inplace=True)

In [64]:
print(df_prepaid_mas_n1.shape)
len(df_prepaid_mas_n1['SUBR_NUM'].unique())

(1757527, 3)


1757527

In [65]:
df_postpaid_mas_n1['PREPOST_TYPE'] = 'PO'
df_prepaid_mas_n1['PREPOST_TYPE'] = 'PR'

## Intersection

In [67]:
post_subr_set = set(df_postpaid_mas_n1['SUBR_NUM'])
pre_subr_set = set(df_prepaid_mas_n1['SUBR_NUM'])

In [68]:
inter = post_subr_set.intersection(pre_subr_set)

In [69]:
df_postpaid_mas_n1[df_postpaid_mas_n1['SUBR_NUM'].isin(inter)]

CUST_NUM  SUBR_NUM    D_HKID_N PREPOST_TYPE
437963  12869011  94312959  C552360(3)           PO

In [72]:
df_prepaid_mas_n1[df_prepaid_mas_n1['SUBR_NUM'].isin(inter)]

CUST_NUM  SUBR_NUM  D_HKID_N PREPOST_TYPE
1199132  55128457  94312959  00000000           PR

## Combine

In [66]:
print(df_postpaid_mas_n1.shape, df_prepaid_mas_n1.shape)

(1036718, 4) (1757527, 4)


In [73]:
df = pd.concat([df_postpaid_mas_n1, df_prepaid_mas_n1], axis=0)
df.shape

(2794245, 4)

## Drop duplicate

In [78]:
df = df[df['CUST_NUM']!='55128457']

In [79]:
df.head()

CUST_NUM  SUBR_NUM    D_HKID_N PREPOST_TYPE
0  12702822  61173398  A843821(0)           PO
1  12703271  98136970    34431093           PO
2  12703949  96618100  Z846193(8)           PO
3  12704980  92063635  D481248(5)           PO
4  12705275  94579115  E769689(6)           PO

## End

## Weblog

In [9]:
l_formatted = getCleansedDomainList(['https://rnr.hk.chinamobile.com', 'https://rnr.hkcsl.com', 'https://rnr.smartone.com'])
print(l_formatted)

['RNR.HK.CHINAMOBILE.COM', 'RNR.HKCSL.COM', 'RNR.SMARTONE.COM']


In [86]:
sdf_tmp = spark.read.parquet('/app/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/date_id=20230104')

In [87]:
sdf_tmp.limit(1).toPandas()

subr_num     ssl_cert_domain  byte_count  hour_binary  total_hit
0  19100006192  GRAPH.FACEBOOK.COM       28658            2          4

In [22]:
print(datetime.now())
df_domain_table = pd.read_sql("""
select 
    /* +PARALLEL(16) */ *
from 
    PRD_BIZ_SUMM_VW.VW_SSL_SED_CALL_SUMM
where
    SSL_CERT_DOMAIN IN ('RNR.HKCSL.COM','RNR.HK.CHINAMOBILE.COM','RNR.SMARTONE.COM','HTTPS://RNR.HKCSL.COM','HTTPS://RNR.HK.CHINAMOBILE.COM','HTTPS://RNR.SMARTONE.COM')
""",con=conn)
print(datetime.now())

2023-01-04 13:53:47.203641
2023-01-04 15:03:43.283074


In [84]:
df_domain_table.head()

SUBR_NUM       MSISDN ACCESS_DATE   SSL_CERT_DOMAIN  BYTE_COUNT  \
0  46264511  85246264511  2022-10-03  RNR.SMARTONE.COM   4153599.0   
1  52361975  85252361975  2022-10-03  RNR.SMARTONE.COM   1831075.0   
2  55048204  85255048204  2022-10-03  RNR.SMARTONE.COM    153477.0   
3  55059714  85255059714  2022-10-03  RNR.SMARTONE.COM     19360.0   
4  55907876  85255907876  2022-10-03  RNR.SMARTONE.COM    454846.0   

      KB_COUNT               HOUR_BINARY  TOTAL_HIT  BYTE_COUNT_UP  \
0  4056.249023  000000000000000001100000         13      2249123.0   
1  1788.159180  000000000100000000000000          7        95768.0   
2   149.879883  000000000000001000000000          3        20827.0   
3    18.906250  000000000001000000000000          3         4914.0   
4   444.185547  000000000000000000000001          6        49066.0   

   KB_COUNT_UP  BYTE_COUNT_DOWN  KB_COUNT_DOWN  SECONDS    MINUTES  
0  2196.409180        1904476.0    1859.839844     2979  49.650000  
1    93.523438        1735307.0    1694.635742     2647  44.116667  
2    20.338867         132650.0     129.541016      363   6.050000  
3     4.798828          14446.0      14.107422       17   0.283333  
4    47.916016         405780.0     396.269531     1823  30.383333

In [ ]:
# sdf_domain_table = spark.sql("""
# select
#     subr_num,
#     domain_index as agg_value,
#     'domain' as agg_level,
#     0 as in_byte,
#     0 as out_byte,
#     byte_count,
#     total_hit,
#     0 as duration,
#     date_id
# from prd_biz_summ_vw.vw_idp_ssl_sed_call_summ
# where date_id between {} and {}
# limit 2
# """.format(20221229, 20221230))

In [ ]:
# # aggregate
# agg_freq = 'daily'
# t_domain_time_key = 'ACCESS_DATE'
# groupby_list = ['SUBR_NUM',t_domain_time_key]
# if agg_freq == 'daily':
#     res_domain=sdf_domain_table.groupby(groupby_list).agg(F.sum('BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),
#                                                           F.sum('TOTAL_HIT').alias('TOTAL_HIT')
#                                                          ).fillna(0).withColumn('APPS',F.lit(app_name))
# else:
#     res_domain=sdf_domain_table.groupby(groupby_list).agg(F.sum('BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),
#                                                           F.sum('TOTAL_HIT').alias('TOTAL_HIT'),
#                                                           F.max('DAYS').alias('MAX_DAYS')
#                                                          ).fillna(0).withColumn('APPS',F.lit(app_name))

In [129]:
# rnr
# df_rnr_kai = pd.read_csv("./rnr_websites.csv")

In [232]:
df_n1_prepost = df.copy()
df_rnr_orcale = df_cust_to_HKID.copy()

In [233]:
print(df_n1_prepost.dtypes)
print(df_rnr_orcale.dtypes)

CUST_NUM        object
SUBR_NUM        object
D_HKID_N        object
PREPOST_TYPE    object
dtype: object
CUST_NUM    object
SUBR_NUM    object
D_HKID_N    object
dtype: object


In [234]:
df_rnr_kai.head(2)

SUBR_NUM       MSISDN ACCESS_DATE         SSL_CERT_DOMAIN  BYTE_COUNT  \
0  52862396  85252862396  2022-09-11  RNR.HK.CHINAMOBILE.COM   1431997.0   
1  64466617  85264466617  2022-09-11  RNR.HK.CHINAMOBILE.COM      6057.0   

      KB_COUNT               HOUR_BINARY  TOTAL_HIT  BYTE_COUNT_UP  \
0  1398.434570  000000000000000001000000         13       844079.0   
1     5.915039  000000000010000000000000          1         2076.0   

   KB_COUNT_UP  BYTE_COUNT_DOWN  KB_COUNT_DOWN  SECONDS   MINUTES  
0   824.295898         587918.0     574.138672      103  1.716667  
1     2.027344           3981.0       3.887695      305  5.083333

In [235]:
df_n1_prepost.head(2)

CUST_NUM  SUBR_NUM    D_HKID_N PREPOST_TYPE
0  12702822  61173398  A843821(0)           PO
1  12703271  98136970    34431093           PO

In [236]:
print(len(df_n1_prepost.SUBR_NUM), len(df_n1_prepost.SUBR_NUM.unique()))

2794244 2794244


In [237]:
df_rnr_orcale.head(2)

CUST_NUM  SUBR_NUM     D_HKID_N
0  41269934  91374003   E248316(9)
1  41643084  60105242  WX030832(1)

In [238]:
print(len(df_rnr_orcale.SUBR_NUM), len(df_rnr_orcale.SUBR_NUM.unique()))

102684 102681


In [239]:
df_orc_n_n1 = pd.merge(df_rnr_orcale, df_n1_prepost, how='left', on=['CUST_NUM', 'SUBR_NUM'])
# df_tmp = df_n1_prepost.join(df_rnr_orcale, on=['SUBR_NUM'], how='left', lsuffix='_n1_prepost', rsuffix='_rnr_orcale')

In [240]:
# join 3 tables
df_rnr_kai['SUBR_NUM'] = df_rnr_kai['SUBR_NUM'].astype(str)
df_rnr_kai_2 = pd.merge(df_rnr_kai, df_rnr_orcale, how='left', on=['SUBR_NUM'], suffixes=('', '_rnr_oracle'))

In [241]:
df_rnr_kai_3 = pd.merge(df_rnr_kai_2, df_n1_prepost, how='left', on=['SUBR_NUM'], suffixes=('_rnr_oracle', '_n1_prepost'))

In [242]:
df_rnr_kai_3.head()

SUBR_NUM          MSISDN ACCESS_DATE         SSL_CERT_DOMAIN  \
0     52862396     85252862396  2022-09-11  RNR.HK.CHINAMOBILE.COM   
1     64466617     85264466617  2022-09-11  RNR.HK.CHINAMOBILE.COM   
2     92000038     85292000038  2022-09-11  RNR.HK.CHINAMOBILE.COM   
3  19101060085  85219101060085  2022-09-11        RNR.SMARTONE.COM   
4  19101080393  85219101080393  2022-09-11        RNR.SMARTONE.COM   

   BYTE_COUNT     KB_COUNT               HOUR_BINARY  TOTAL_HIT  \
0   1431997.0  1398.434570  000000000000000001000000         13   
1      6057.0     5.915039  000000000010000000000000          1   
2   3147157.0  3073.395508  000000000000000000100000         22   
3   4106365.0  4010.122070  000000000100000000000000          7   
4   1842231.0  1799.053711  000000000000000000000100          6   

   BYTE_COUNT_UP  KB_COUNT_UP  BYTE_COUNT_DOWN  KB_COUNT_DOWN  SECONDS  \
0       844079.0   824.295898         587918.0     574.138672      103   
1         2076.0     2.027344           3981.0       3.887695      305   
2      1205608.0  1177.351562        1941549.0    1896.043945     2250   
3      1384930.0  1352.470703        2721435.0    2657.651367     1987   
4       105489.0   103.016602        1736742.0    1696.037109      262   

     MINUTES CUST_NUM_rnr_oracle D_HKID_N_rnr_oracle CUST_NUM_n1_prepost  \
0   1.716667                 NaN                 NaN            08620092   
1   5.083333                 NaN                 NaN                 NaN   
2  37.500000                 NaN                 NaN            11998673   
3  33.116667                 NaN                 NaN                 NaN   
4   4.366667                 NaN                 NaN                 NaN   

  D_HKID_N_n1_prepost PREPOST_TYPE  
0          Z258049(8)           PO  
1                 NaN          NaN  
2          Z852417(4)           PO  
3                 NaN          NaN  
4                 NaN          NaN

In [215]:
df_rnr_kai_3[~df_rnr_kai_3['CUST_NUM_rnr_oracle'].isna() & df_rnr_kai_3['CUST_NUM_n1_prepost'].isna()].shape

(5330, 19)

In [207]:
df_rnr_kai_3[df_rnr_kai_3['SUBR_NUM'].isin(l_multi_hkid)]

SUBR_NUM       MSISDN ACCESS_DATE         SSL_CERT_DOMAIN  BYTE_COUNT  \
16248  91346954  85291346954  2022-09-24  RNR.HK.CHINAMOBILE.COM    850031.0   
16249  91346954  85291346954  2022-09-24  RNR.HK.CHINAMOBILE.COM    850031.0   
45966  95118258  85295118258  2022-10-21           RNR.HKCSL.COM   1718537.0   
45967  95118258  85295118258  2022-10-21           RNR.HKCSL.COM   1718537.0   
55609  94503687  85294503687  2022-10-27        RNR.SMARTONE.COM    844777.0   
55610  94503687  85294503687  2022-10-27        RNR.SMARTONE.COM    844777.0   
63088  94503687  85294503687  2022-10-31        RNR.SMARTONE.COM    710486.0   
63089  94503687  85294503687  2022-10-31        RNR.SMARTONE.COM    710486.0   
77935  91917573  85291917573  2022-11-11        RNR.SMARTONE.COM    166026.0   
77936  91917573  85291917573  2022-11-11        RNR.SMARTONE.COM    166026.0   
79672  91917573  85291917573  2022-11-12        RNR.SMARTONE.COM     27882.0   
79673  91917573  85291917573  2022-11-12        RNR.SMARTONE.COM     27882.0   

          KB_COUNT               HOUR_BINARY  TOTAL_HIT  BYTE_COUNT_UP  \
16248   830.108398  000000000000010000000000          6        62717.0   
16249   830.108398  000000000000010000000000          6        62717.0   
45966  1678.258789  000000000000000100000000         19       142334.0   
45967  1678.258789  000000000000000100000000         19       142334.0   
55609   824.977539  000000000001000000000000          5        55756.0   
55610   824.977539  000000000001000000000000          5        55756.0   
63088   693.833984  000000000000000010000000          7        57706.0   
63089   693.833984  000000000000000010000000          7        57706.0   
77935   162.134766  000000000000000110000000        131       120832.0   
77936   162.134766  000000000000000110000000        131       120832.0   
79672    27.228516  000000000001000000000000         19        20694.0   
79673    27.228516  000000000001000000000000         19        20694.0   

       KB_COUNT_UP  BYTE_COUNT_DOWN  KB_COUNT_DOWN  SECONDS    MINUTES  \
16248    61.247070         787314.0     768.861328      537   8.950000   
16249    61.247070         787314.0     768.861328      537   8.950000   
45966   138.998047        1576203.0    1539.260742      145   2.416667   
45967   138.998047        1576203.0    1539.260742      145   2.416667   
55609    54.449219         789021.0     770.528320      486   8.100000   
55610    54.449219         789021.0     770.528320      486   8.100000   
63088    56.353516         652780.0     637.480469      195   3.250000   
63089    56.353516         652780.0     637.480469      195   3.250000   
77935   118.000000          45194.0      44.134766     2728  45.466667   
77936   118.000000          45194.0      44.134766     2728  45.466667   
79672    20.208984           7188.0       7.019531      405   6.750000   
79673    20.208984           7188.0       7.019531      405   6.750000   

      CUST_NUM_rnr_oracle D_HKID_N_rnr_oracle CUST_NUM_n1_prepost  \
16248            55253654           P6561970B            55253654   
16249            08497741          P033789(0)            55253654   
45966            06662321          Z305875(2)            12839677   
45967            12839677          Z338457(9)            12839677   
55609            55523753            6401953B            55523753   
55610                              W614931(9)            55523753   
63088            55523753            6401953B            55523753   
63089                              W614931(9)            55523753   
77935            55594365          P365422(6)            55594365   
77936            11730105          R303525(4)            55594365   
79672            55594365          P365422(6)            55594365   
79673            11730105          R303525(4)            55594365   

      D_HKID_N_n1_prepost PREPOST_TYPE  
16248            00000000           PR  
16249            00000000           PR  
45966          Z338

In [246]:
df_rnr_kai_3.to_csv('./rnr_websites_mapped3.csv', index=False)

In [167]:
df_n1_prepost.head(20)

CUST_NUM  SUBR_NUM    D_HKID_N PREPOST_TYPE
0   12702822  61173398  A843821(0)           PO
1   12703271  98136970    34431093           PO
2   12703949  96618100  Z846193(8)           PO
3   12704980  92063635  D481248(5)           PO
4   12705275  94579115  E769689(6)           PO
5   12677119  98885112  Z671091(4)           PO
6   08283129  61110862  Z325152(8)           PO
7   08851145  94976811  G049135(6)           PO
8   03145333  60118257  Z464059(5)           PO
9   12393847  91430812  K055416(2)           PO
10  11418330  90902435  Y430059(7)           PO
11  12311239  61124398  K523754(8)           PO
12  10608719  96881086  K257142(0)           PO
13  08634391  93801906  B634382(7)           PO
14  07878208  92512284  K286372(3)           PO
15  09048631  55088301  C494549(0)           PO
16  04017074  65760116  G013845(1)           PO
17  04432358  92281460  Z337148(5)           PO
18  04437924  90462382  D676208(6)           PO
19  04516871  96870047  D704010(6)           PO

In [9]:
# rnr
sdf_rnr = spark.read.option("header", True).csv("./rnr_websites.csv")

In [10]:
sdf_rnr.limit(5).toPandas()

SUBR_NUM          MSISDN ACCESS_DATE         SSL_CERT_DOMAIN BYTE_COUNT  \
0     52862396     85252862396  2022-09-11  RNR.HK.CHINAMOBILE.COM  1431997.0   
1     64466617     85264466617  2022-09-11  RNR.HK.CHINAMOBILE.COM     6057.0   
2     92000038     85292000038  2022-09-11  RNR.HK.CHINAMOBILE.COM  3147157.0   
3  19101060085  85219101060085  2022-09-11        RNR.SMARTONE.COM  4106365.0   
4  19101080393  85219101080393  2022-09-11        RNR.SMARTONE.COM  1842231.0   

          KB_COUNT               HOUR_BINARY TOTAL_HIT BYTE_COUNT_UP  \
0  1398.4345703125  000000000000000001000000        13      844079.0   
1     5.9150390625  000000000010000000000000         1        2076.0   
2  3073.3955078125  000000000000000000100000        22     1205608.0   
3  4010.1220703125  000000000100000000000000         7     1384930.0   
4  1799.0537109375  000000000000000000000100         6      105489.0   

      KB_COUNT_UP BYTE_COUNT_DOWN    KB_COUNT_DOWN SECONDS             MINUTES  
0  824.2958984375        587918.0    574.138671875     103  1.7166666666666666  
1      2.02734375          3981.0     3.8876953125     305   5.083333333333333  
2    1177.3515625       1941549.0  1896.0439453125    2250                37.5  
3  1352.470703125       2721435.0  2657.6513671875    1987   33.11666666666667  
4  103.0166015625       1736742.0   1696.037109375     262   4.366666666666666

In [48]:
sdf_output = sdf_rnr.join(sdf, 'SUBR_NUM', 'left').join()

In [49]:
sdf_output.limit(5).toPandas()

SUBR_NUM          MSISDN ACCESS_DATE   SSL_CERT_DOMAIN BYTE_COUNT  \
0  19100148960  85219100148960  2022-10-15  RNR.SMARTONE.COM    56982.0   
1  19101023243  85219101023243  2022-12-11  RNR.SMARTONE.COM    16605.0   
2  19101044549  85219101044549  2022-12-10  RNR.SMARTONE.COM   580614.0   
3  19101089404  85219101089404  2022-11-02  RNR.SMARTONE.COM   581425.0   
4  19101096482  85219101096482  2022-10-16  RNR.SMARTONE.COM     1841.0   

         KB_COUNT               HOUR_BINARY TOTAL_HIT BYTE_COUNT_UP  \
0    55.646484375  000000000000100000000000         2        9560.0   
1   16.2158203125  000000000001000000000000         1        5255.0   
2   567.005859375  000000000010000000000000         5       27916.0   
3  567.7978515625  000000000000000010000000         3       28777.0   
4    1.7978515625  000000000000010000000000         1        1168.0   

     KB_COUNT_UP BYTE_COUNT_DOWN  KB_COUNT_DOWN SECONDS             MINUTES  \
0      9.3359375         47422.0   46.310546875     613  10.216666666666667   
1   5.1318359375         11350.0   11.083984375     427   7.116666666666666   
2    27.26171875        552698.0  539.744140625     316   5.266666666666667   
3  28.1025390625        552648.0    539.6953125     192                 3.2   
4       1.140625           673.0   0.6572265625      77  1.2833333333333334   

  CUST_NUM D_HKID_N PREPOST_TYPE  
0     None     None         None  
1     None     None         None  
2     None     None         None  
3     None     None         None  
4     None     None         None

In [60]:
# sdf_output.write.option("header",True).coalesce(1).\
#  .csv("./rnr_websites_mapped.csv")

sdf_output.toPandas().to_csv("./rnr_websites_mapped2.csv", index=False)

In [69]:
sdf.count()

2791647

In [70]:
sdf.limit(2).toPandas()

CUST_NUM  SUBR_NUM    D_HKID_N PREPOST_TYPE
0  08790141  93145780  P027771(5)           PO
1  06193251  97777698  K777425(7)           PO

In [78]:
# filter_table = sdf.select(
#     'CUST_NUM',
#     'SUBR_NUM',
#     'D_HKID_N',	
# )

filter_table = sdf

filter_table.limit(1).toPandas()

CUST_NUM  SUBR_NUM    D_HKID_N
0  08790141  93145780  P027771(5)

In [121]:
filter_table = sdf

## Get doamin summary

In [62]:
val_agg_period_start, val_agg_period_end = 20221013, 20221211
t_domain_table='t_domain_daily'
t_domain_time_key='date_id'

In [63]:
# chinamobile
l_raw = ['https://rnr.hk.chinamobile.com',
'https://rnr.hkcsl.com',
'https://rnr.smartone.com']

## From raw table

In [108]:
spark.sql("""
select
    *
from prd_biz_summ_vw.vw_idp_ssl_sed_call_summ
where date_id between {from_date_id} and {to_date_id}
""".format(
    from_date_id=20221013, to_date_id=20221211
)).cache().createOrReplaceTempView("t_domain_daily_raw")

In [109]:
spark.table("t_domain_daily_raw").limit(5).toPandas()

subr_num                    ssl_cert_domain  byte_count  hour_binary  \
0  19100005771                       DNS.JOOX.COM       19504        18432   
1     98897557  AUCTION-LOAD.UNITYADS.UNITY3D.COM       17045      8388608   
2  19100007216          SUGGESTQUERIES.GOOGLE.COM       72901        98326   
3     98897557            SAFEBROWSING.GOOGLE.COM      137054      8798475   
4  19100009972                  FONTS.GSTATIC.COM       49762         8192   

   total_hit   date_id  
0          2  20221018  
1          1  20221018  
2          7  20221018  
3         13  20221018  
4          1  20221018

In [117]:
sdf_domain_table = spark.table("t_domain_daily_raw")

In [112]:
l_formatted = getCleansedDomainList(['https://rnr.hk.chinamobile.com', 'https://rnr.hkcsl.com', 'https://rnr.smartone.com'])
print(l_formatted)

['RNR.HK.CHINAMOBILE.COM', 'RNR.HKCSL.COM', 'RNR.SMARTONE.COM']


In [119]:
# 
agg_value_table_cmhk = spark.createDataFrame(pd.DataFrame(['RNR.HK.CHINAMOBILE.COM'], columns=['ssl_cert_domain']))
sdf_domain_table_cmhk = sdf_domain_table.join(agg_value_table_cmhk,sdf_domain_table.ssl_cert_domain==agg_value_table_cmhk.ssl_cert_domain,'inner').select(sdf_domain_table_cmhk['*'])

In [120]:
sdf_domain_table_cmhk.count()

8066

In [122]:
for col in sdf_domain_table_cmhk.columns:
    sdf_domain_table_cmhk=sdf_domain_table_cmhk.withColumnRenamed(col, col.upper())

# filter table
sdf_domain_table_cmhk=sdf_domain_table_cmhk.join(filter_table,sdf_domain_table_cmhk.SUBR_NUM==filter_table.SUBR_NUM,'inner').select(sdf_domain_table_cmhk['*'])

groupby_list= ['SUBR_NUM','date_id']

# daily groupby
res_domain_cmhk=sdf_domain_table_cmhk.groupby(groupby_list).agg(F.sum('BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),
                                                      F.sum('TOTAL_HIT').alias('TOTAL_HIT')
                                                     ).fillna(0).withColumn('APPS',F.lit('CMHK'))

In [123]:
res_domain_cmhk.limit(10).toPandas()

KeyboardInterrupt: 

## From t domain daily

In [91]:
# china mobile
summary_cmhk = getDomainSummary(agg_value_table=getAggValue(['https://rnr.hk.chinamobile.com']),
                                  agg_freq='daily',
                                  agg_period_start=val_agg_period_start,
                                  agg_period_end=val_agg_period_end,
                                  app_name='PRE_POST_CMHK',
                                  filter_table=filter_table).cache()

In [92]:
summary_cmhk.count()

0

In [72]:
summary_cmhk.limit(1).toPandas()

Empty DataFrame
Columns: [SUBR_NUM, date_id, TOTAL_BYTE_COUNT, TOTAL_HIT, APPS]
Index: []

In [67]:
# csl
summary_csl = getDomainSummary(agg_value_table=getAggValue(['https://rnr.hkcsl.com']),
                                  agg_freq='daily',
                                  agg_period_start=val_agg_period_start,
                                  agg_period_end=val_agg_period_end,
                                  app_name='PRE_POST_CSL',
                                  filter_table=filter_table).cache()

In [73]:
summary_csl.count()

0

In [68]:
# smt
summary_smt = getDomainSummary(agg_value_table=getAggValue(['https://rnr.smartone.com']),
                                  agg_freq='daily',
                                  agg_period_start=val_agg_period_start,
                                  agg_period_end=val_agg_period_end,
                                  app_name='PRE_POST_SMT',
                                  filter_table=filter_table).cache()

In [74]:
summary_smt.count()

0

In [75]:
# test
summary_test = getDomainSummary(agg_value_table=getAggValue(['https://www.smartone.com/']),
                                  agg_freq='daily',
                                  agg_period_start=val_agg_period_start,
                                  agg_period_end=val_agg_period_end,
                                  app_name='PRE_POST_TEST',
                                  filter_table=filter_table).cache()

In [76]:
summary_test.count()

154207

In [90]:
# but why????
getAggValue(['https://rnr.hk.chinamobile.com', 'https://rnr.hkcsl.com', 'https://rnr.smartone.com']).toPandas()

agg_value
0     903320
1     903116
2     880645

In [80]:
temp = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_daily/date_id=20221211')

In [82]:
temp.limit(20).toPandas()

subr_num  agg_value agg_level  in_byte  out_byte  byte_count  \
0   19100007657      20104    domain        0         0       15853   
1      96681026      27665    domain        0         0      826614   
2   19100025837      39164    domain        0         0     3436600   
3      96681026      44987    domain        0         0      746095   
4   19100025837      48354    domain        0         0       65670   
5      96681029      20201    domain        0         0       18611   
6   19100026228      56221    domain        0         0       46287   
7      96681029      64947    domain        0         0       15598   
8   19100032412      15783    domain        0         0       20397   
9      96681034      50320    domain        0         0       12457   
10  19100032412      81708    domain        0         0      523985   
11     96681038     115461    domain        0         0        6014   
12  19100050672      31627    domain        0         0     1463697   
13     96681050       8116    domain        0         0       58646   
14  19100073015      10629    domain        0         0      320023   
15     96681050      22305    domain        0         0       39042   
16  19100073015      29425    domain        0         0       66689   
17     96681056      66565    domain        0         0   121265457   
18  19100073015      15614    domain        0         0       22016   
19     96681057      59587    domain        0         0      260375   

    total_hit  duration  
0           2         0  
1          93         0  
2           4         0  
3           1         0  
4           6         0  
5           2         0  
6           2         0  
7           2         0  
8           2         0  
9           1         0  
10         23         0  
11          1         0  
12          4         0  
13          6         0  
14         45         0  
15          3         0  
16          6         0  
17          4         0  
18          2         0  
19          1         0

## Pivot and Merge

In [ ]:
res_merged=summary_cmhk.union(summary_csl).union(summary_smt).cache()

In [ ]:
res_merged.limit(2).show()

In [ ]:
res_pivot=res_merged.groupBy('date_id','SUBR_NUM')\
                    .pivot('APPS').agg(f.sum('TOTAL_BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),\
                                       f.countDistinct('date_id').alias('TOTAL_DAYS')\
                                      ).na.fill(0).withColumnRenamed('date_id','DATE_ID')

## Result

In [ ]:
sdf_output = res_pivot.join(sdf,['SUBR_NUM'],'left')

In [ ]:
sdf_output.limit(2).toPandas()

In [ ]:
sdf_output.select(
    col('DATE_ID').alias('DATE')\
    ,'CUST_NUM', 'SUBR_NUM'\
    ,col('HKBN').alias('HKBN_RET_HOTLINE_IN_MINS')\
    ,col('NETVIGATOR').alias('NETVIGATOR_RET_HOTLINE_IN_MINS')\
    ,col('hkbn_TOTAL_BYTE_COUNT').alias('HKBN_USAGE')\
    ,col('house_moving_TOTAL_BYTE_COUNT').alias('HOUSE_MOVING_USAGE')\
    ,col('house_renovating_companys_TOTAL_BYTE_COUNT').alias('HOUSE_RENOVATING_COMPANYS_USAGE')\
    ,col('netvigator_TOTAL_BYTE_COUNT').alias('NETVIGATOR_USAGE')\
    ,col('village_TOTAL_BYTE_COUNT').alias('VILLAGE_PROPERTY_AGENT_USAGE')\
    ,col('LANG').alias('LANGUAGE')\
    ).toPandas().to_csv('./prepost_weblog_20221212.csv', index=False)

In [ ]:
spark.stop()
conn.close()
sc.stop()

## Backup

In [13]:
df_summary = summary.toPandas()

In [58]:
df_summary.to_csv('summary.csv', index=False)
# save to parquet
# summary.write.mode('append').parquet(f'/app/ws-simon/projects/football_lover_20221111/data/')

In [ ]:
# df = df_summary[df_summary['TOTAL_BYTE_COUNT'] >= ]

In [41]:
fb_list = list(map(str, df_summary['SUBR_NUM']))

In [42]:
fb_list[0]

'92292817'

In [48]:
df2 = source_table.copy()
df2['SEGMENT'] = ['football_lover' if str(r['SUBR_NUM']) in fb_list else 'gamer' if r['GAME_LOVER_TOTAL_DAY_CNT'] >= 27 else 'streaming_lover' if r['MUSIC_LOVER_TOTAL_DAY_CNT'] >= 18 else 'na' for i, r in df2.iterrows()]

In [49]:
df_output = df2[['CUST_NUM', 'SUBR_NUM', 'SEGMENT']]
df_output = df_output[df_output['SEGMENT'] != 'na']
df_output.to_csv('./football_lover.csv', index=False)

In [2]:
/0

TypeError: 'int' object is not callable

In [ ]:
def ggetDomainSummary(agg_value_table:object, agg_freq:str, agg_period_start:str, agg_period_end:str,app_name:str, filter_table=None):
    """
    Aggregate following Weblog Features with Web Domain Agg Value
        -  Sum of Data Usage in byte 
        -  Maximum of Total Hit
        -  Maximum of Day Used

    :param agg_value_table: Web Domain Agg Value Table
    :param agg_freq: Aggeration Frequncy ('daily','weekly','monthly')
    :param agg_period_start: Aggreate period start time in following format:
                                - daily: yyyymmdd
                                - weekly: yyyymmdd
                                - monthly: yyyymm
    :param agg_period_end: Aggreate period end time with same format of agg_period_start
    :param app_name: Column of Application name in the output 
    :param filter_table: Optional, inner join table on `SUBR_NUM` 

    :returns: Temp View of Aggerate Domain Summary:
            
    :raises ValueError: When You have input wrong agg_freq
    
    TODO:
    1. Add CUST_NUM from filter_table if it exists.
    """

    if int(agg_period_start) > int(agg_period_end):
            correct_agg_period_end=agg_period_start
            agg_period_start=agg_period_end
            agg_period_end=correct_agg_period_end

    if agg_freq == 'daily':
            t_domain_table='t_domain_daily'
            t_domain_time_key='date_id'
    elif agg_freq == 'weekly':
        t_domain_table='t_domain_weekly'
        t_domain_time_key='week_id'
    elif agg_freq == 'monthly':
        t_domain_table='t_domain_monthly'
        t_domain_time_key='month_id'
    else:
        raise ValueError("Wrong agg_freq received. Please input correct agg_freq (daily, weekly, monthly)")

    sdf_domain_table=spark.read.parquet(f"/app/ws-ic/hive/weblog.db/{t_domain_table}").where(f"{t_domain_time_key} between {agg_period_start} and  {agg_period_end}")

    sdf_domain_table=sdf_domain_table.join(agg_value_table,sdf_domain_table.agg_value==agg_value_table.agg_value,'inner').select(sdf_domain_table['*'])
    
    for col in sdf_domain_table.columns:
        sdf_domain_table=sdf_domain_table.withColumnRenamed(col, col.upper())
    
    if filter_table:
        sdf_domain_table=sdf_domain_table.join(filter_table,sdf_domain_table.SUBR_NUM==filter_table.SUBR_NUM,'inner').select(sdf_domain_table['*'])

    groupby_list= ['SUBR_NUM',t_domain_time_key]
    if agg_freq == 'daily':
        res_domain=sdf_domain_table.groupby(groupby_list).agg(F.sum('BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),
                                                              F.sum('TOTAL_HIT').alias('TOTAL_HIT')
                                                             ).fillna(0).withColumn('APPS',F.lit(app_name))
    else:
        res_domain=sdf_domain_table.groupby(groupby_list).agg(F.sum('BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),
                                                              F.sum('TOTAL_HIT').alias('TOTAL_HIT'),
                                                              F.max('DAYS').alias('MAX_DAYS')
                                                             ).fillna(0).withColumn('APPS',F.lit(app_name))
